In [132]:
# Prerequisites, just run this
import networkx as nx
import pandas as pd
import json
from ipycytoscape import *
df = pd.read_csv('data_materialcheck.csv', sep=';')

In [133]:
# Also this
df = df.fillna('')
df

,id,type,label,shape,fillcolor,fontcolor,rank,subgraph,connection_from,connection_to,edgeLabel
0,1,lower,Project Start,rectangle,Purple,white,1,initial,Project Start,Type of Project,
1,2,lower,Type of Project,diamond,Thistle,white,1,initial,Type of Project,Type of Study,Continue after choice
2,3,lower,Type of Study,diamond,Thistle,white,1,initial,Type of Study,Data Availability,Empircal
3,4,lower,Data Availability,diamond,Thistle,white,1,initial,Data Availability,Data Simulation,
4,5,lower,Data Simulation,diamond,Thistle,white,1,initial,Data Simulation,Experiment Design,
5,6,lower,Experiment Design,rectangle,Purple,white,1,initial,Experiment Design,Data Collection,
6,7,lower,Data Collection,rectangle,Purple,white,1,initial,Data Collection,Data Validation,
7,8,lower,Data Validation,rectangle,Purple,white,1,initial,Data Validation,Data Analysis,
8,9,lower,Data Analysis,rectangle,Purple,white,1,initial,Data Analysis,Writing,
9,10,lower,Writing,rectangle,Purple,white,1,initial,Writing,Dissemination,


In [134]:
# Import stuff to dynamically update the graph
from ipywidgets import Output
from IPython.display import display

# instantiate an jupyternotebook output
out = Output()
# style
fin_style = []
# define a function that draws the network wrt a selected node
def draw_net(node_filter): 
    base_graph = nx.Graph() # reset the network obj
    for index, row in df.iterrows(): # iterate the dataframe
        if row['subgraph'] == node_filter.replace(' ',''): # if the clicked node has a subgraph
            base_graph.add_node(row['connection_to']) # add node
            base_graph.add_node(row['connection_from']) # add node
            base_graph.add_edge(row['connection_to'], row['connection_from']) # now connect the nodes
            rank = row['rank'] # also use rank as output
            subgraph = row['subgraph'] # also use subgraph as output
        elif row['label'] == node_filter: # if the clicked node does not have a subgraph, but it is alone
            base_graph.add_node(row['label']) # just draw the node on its own
            rank = row['rank'] # also use rank as output
            subgraph = row['subgraph'] # also use subgraph as output
    return [base_graph,rank,subgraph] # return the drawn graph
# define a function that logs mouse clicks
def log_clicks(node):
    # style
    fin_style = []
    with out: # while the graph is drawn
        [net,rank,subgraph] = draw_net(node['data']['id']) # draw the network
        cytoscapeobj.graph.clear() # reset cytoscape object
        cytoscapeobj.graph.add_graph_from_networkx(net, directed=True) # now convert the network to a cytoscape object
        cytoscapeobj.set_layout(name = 'dagre')  # it is hierarchical, so show it like that!
        for node in net.nodes: # now iterate the nodes
            shape = df[df.eval("(label == \""+node+"\") & ("+"subgraph == \""+subgraph+"\") & (rank == "+str(rank)+")")]['shape'].values # get the shape by filtering the dataframe
            fillcolor = df[df.eval("(label == \""+node+"\")")]['fillcolor'].values
            if len(shape) == 0: # if not found, i.e. 0th level node
                shape = ['rectangle'] # draw as rectangle
            if len(fillcolor) == 0:
                fillcolor = ['Gray']
            node_dict = {'selector': f'node[id = \"{node}\"]'} # get node id in cyto logic
            style_dict = {"style": {'font-family': 'helvetica',
                                    'font-size': '20px',
                                    'label': node, 
                                    'shape': shape[0],
                                    'background-color': fillcolor[0]
                                   }} # put node id and shape in cyto logic
            node_dict.update(style_dict) # update dict
            fin_style.append(node_dict) # append to style list
        cytoscapeobj.set_style(fin_style) # now set the final style
        
# just initialise the 0th level graph
def init_graph():
    base_graph = nx.Graph() # reset the network obj
    fin_style = []
    for index, row in df.iterrows(): # iterate the dataframe
        if row['subgraph'] == 'initial' and row['rank'] == 1: # only draw the initial and 0th level nodes
            base_graph.add_edge(row['connection_to'], row['connection_from']) # add the edges, nodes should be added automatically
            node = row['label']
            node_dict = {'selector': f'node[id = \"{node}\"]'} # get node id in cyto logic
            style_dict = {"style": {'font-family': 'helvetica',
                                    'font-size': '20px',
                                    'label': row['label'], 
                                    'shape': row['shape'],
                                    'background-color': row['fillcolor']
                                   }} # put node id and shape in cyto logic
            node_dict.update(style_dict) # update dict
            fin_style.append(node_dict) # append to style list
    cytoscapeobj.graph.clear() # reset cytoscape object
    cytoscapeobj.graph.add_graph_from_networkx(base_graph, directed=True) # now convert the network to a cytoscape object
    cytoscapeobj.set_layout(name = 'circle') # show it in a circular fashion
    cytoscapeobj.set_style(fin_style) # set the style
    
# reset the graph
def res_graph(node):
    with out: # reset the DRAWN graph
        base_graph = nx.Graph() # reset the network obj
        fin_style = []
        for index, row in df.iterrows(): # iterate the dataframe
            if row['subgraph'] == 'initial' and row['rank'] == 1: # only draw the initial and 0th level nodes 
                base_graph.add_edge(row['connection_to'], row['connection_from']) # add the edges, nodes should be added automatically
                node = row['label']
                node_dict = {'selector': f'node[id = \"{node}\"]'} # get node id in cyto logic
                style_dict = {"style": {'font-family': 'helvetica',
                                        'font-size': '20px',
                                        'label': row['label'], 
                                        'shape': row['shape'],
                                        'background-color': row['fillcolor']
                                       }} # put node id and shape in cyto logic
                node_dict.update(style_dict) # update dict
                fin_style.append(node_dict) # append to style list
        cytoscapeobj.graph.clear() # reset cytoscape object
        cytoscapeobj.graph.add_graph_from_networkx(base_graph, directed=True) # now convert the network to a cytoscape object
        cytoscapeobj.set_layout(name = 'circle') # show it in a circular fashion
        cytoscapeobj.set_style(fin_style) # set the style
    
cytoscapeobj = CytoscapeWidget() # ok now create the cytoscape object
my_style = [ # a variable for our initial style
    {'selector': 'node','style': {
        'font-family': 'helvetica',
        'font-size': '20px',
        'label': 'data(id)', 
        'shape':'rectangle'}},
    ]
init_graph() # init the network graph
#cytoscapeobj.set_style(my_style) # set the style
#
cytoscapeobj.on('node', 'click', log_clicks) # dynamically listen to left clicks
cytoscapeobj.on('node', 'cxttap', res_graph) # dynamically listen to right clicks
display(cytoscapeobj) # display the object
display(out) # display the output

CytoscapeWidget(cytoscape_layout={'name': 'circle'}, cytoscape_style=[{'selector': 'node[id = "Project Start"]…

Output()

In [127]:
cytoscapeobj.get_style()

[{'selector': 'node[id = "Project Start"]',
  'style': {'font-family': 'helvetica',
   'font-size': '20px',
   'label': 'Project Start',
   'shape': 'rectangle',
   'background-color': 'Purple'}},
 {'selector': 'node[id = "Type of Project"]',
  'style': {'font-family': 'helvetica',
   'font-size': '20px',
   'label': 'Type of Project',
   'shape': 'diamond',
   'background-color': 'Thistle'}},
 {'selector': 'node[id = "Type of Study"]',
  'style': {'font-family': 'helvetica',
   'font-size': '20px',
   'label': 'Type of Study',
   'shape': 'diamond',
   'background-color': 'Thistle'}},
 {'selector': 'node[id = "Data Availability"]',
  'style': {'font-family': 'helvetica',
   'font-size': '20px',
   'label': 'Data Availability',
   'shape': 'diamond',
   'background-color': 'Thistle'}},
 {'selector': 'node[id = "Data Simulation"]',
  'style': {'font-family': 'helvetica',
   'font-size': '20px',
   'label': 'Data Simulation',
   'shape': 'diamond',
   'background-color': 'Thistle'}},
 {'

In [128]:
{'selector': 'node[id = \"{node["id"]}\"]'}

{'selector': 'node[id = "{node["id"]}"]'}

In [129]:
id = 0
for edge in cytoscapeobj.graph.edges:
    edge.data['id'] = df['edgeLabel'][id]
    id += 1